In [1]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.
You have access to two tools: 

- get_weather_for_location : use this tool to get the current weather of the provided location
- get_user_location : use this tool to get thr current location of the user

If a user asks you for the weather, make sure you know the location. 
If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location.
"""

In [4]:
from langchain.tools import tool, ToolRuntime
from dataclasses import dataclass

@tool
def get_weather_for_location(location:str)->str:
    """Get the current weather of the location"""
    return f"The weather in {location} is windy and sunny"

@dataclass
class Context:
    """Custom runtime context schema"""
    user_name: str

@tool
def get_user_location(runtime:ToolRuntime[Context])->str:
    """Retreive user location based on user name"""
    user_name = runtime.context.user_name
    return "Kochi" if  user_name == "sachin" else "New Delhi"


In [18]:
# Setup the model
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-5-nano",
    temperature= 0.5, # deicde the randomeness of the llm output,  the hiher the value, more creative the output will be
    max_tokens= 1000, 
    timeout=10

)

llm.invoke("hi")


AIMessage(content='Hi there! How can I help today? Tell me what you’re working on or what you’d like to do. I can explain topics, draft or edit writing, help with coding, solve math problems, brainstorm ideas, plan a project, or just chat. What would you like to start with?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 326, 'prompt_tokens': 7, 'total_tokens': 333, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D0V6Jz1lGNAR7FqH945yV3ryqC3hI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019be153-c436-7a71-92f5-b858860d5e7a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 326, 'total_tokens'

In [20]:
# structured responnse using dataclass
from dataclasses import dataclass

@dataclass
class ResponseFormat:
    """Response Schema for the agent"""
    punny_response:str
    weather_condition:str|None  = None


In [21]:
# add memory to the agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [29]:
# lets create the agent
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=llm,
    system_prompt=SYSTEM_PROMPT,
    tools =[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer

)

In [33]:
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke({"messages":[{"role": "user", "content": "What is weather in Kochi?"}]}, 
                        config=config,
                        context=Context(user_name="sachin"))
response

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_5Q24NroIwaLRRF1U038COheo", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

In [39]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure model
model = init_chat_model(
    "gpt-5",
    temperature=0
)

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}


In [40]:

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )



# )

KeyboardInterrupt: 